In [1]:
from timeit import timeit
from IPython.display import display
import matplotlib
# matplotlib.use("pgf")
from matplotlib import pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from collections import Counter
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import cohen_kappa_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from joblib import dump, load
from imblearn.under_sampling import RandomUnderSampler
import shap
import seaborn as sns

In [2]:
from compiledataset import load_dataset, compile_dataset

PATH = "/home/hampus/miun/master_thesis/Datasets"

datasets = {}

# dataset: pd.DataFrame = load_dataset(PATH + "/ORNL", "data_a.csv")
# dataset["remarks"] = "No DLC available"
# datasets["ROAD"] = dataset.to_dict("records")

dataset: pd.DataFrame = load_dataset(PATH + "/Survival", "data.csv")
dataset["remarks"] = "-"
datasets["Survival"] = dataset.to_dict("records")

# dataset: pd.DataFrame = load_dataset(PATH + "/Hisingen", "data.csv")
# dataset["remarks"] = "-"
# datasets["Hisingen"] = dataset.to_dict("records")


df = compile_dataset(datasets)

dataset = None # Release memory, as it isn't used for now
datasets = None

display(df)

2022-03-08 18:52:49.330772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 18:52:49.331156: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hampus/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-08 18:52:49.331218: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hampus/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-03-08 18:52:49.331266: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcu

,t,ID,DLC,d0,d1,d2,d3,d4,d5,d6,d7,dt,dt_ID,ones,ones_ID,dataset,type,Label
0,0.000000,608,8,21,30,30,48,0,143,123,44,0.000444,0.009978,27,3,Survival,none,0
1,0.000232,672,8,64,0,118,29,204,4,227,0,0.000232,0.009977,20,3,Survival,none,0
2,0.000480,704,8,20,0,0,0,0,0,0,0,0.000248,0.009981,2,3,Survival,none,0
3,0.000714,809,8,12,185,127,20,17,32,0,20,0.000234,0.009978,21,5,Survival,none,0
4,0.000955,1349,8,216,0,0,140,0,0,0,0,0.000241,0.009978,7,5,Survival,none,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735835,59.573580,562,8,0,0,0,8,0,0,0,0,0.000248,0.020620,1,4,Survival,none,0
1735836,59.573815,398,8,0,0,0,105,166,154,6,154,0.000235,0.013305,18,5,Survival,none,0
1735837,59.574057,401,8,6,154,6,170,6,154,0,0,0.000242,0.013092,18,4,Survival,none,0
1735838,59.574492,199,4,0,7,2,32,0,0,0,0,0.000435,0.012329,5,5,Survival,none,0


In [3]:
df.drop(columns=["d0", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "ID", "DLC", "t"], inplace=True, errors="ignore")

display(df)

,dt,dt_ID,ones,ones_ID,dataset,type,Label
0,0.000444,0.009978,27,3,Survival,none,0
1,0.000232,0.009977,20,3,Survival,none,0
2,0.000248,0.009981,2,3,Survival,none,0
3,0.000234,0.009978,21,5,Survival,none,0
4,0.000241,0.009978,7,5,Survival,none,0
...,...,...,...,...,...,...,...
1735835,0.000248,0.020620,1,4,Survival,none,0
1735836,0.000235,0.013305,18,5,Survival,none,0
1735837,0.000242,0.013092,18,4,Survival,none,0
1735838,0.000435,0.012329,5,5,Survival,none,0


In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
df.loc[:, (df.columns != "type") & (df.columns != "dataset")] = MinMaxScaler(copy=True).fit_transform(df.loc[:, (df.columns != "type") & (df.columns != "dataset")])

display(df)

,dt,dt_ID,ones,ones_ID,dataset,type,Label
0,0.046644,0.000242,0.529412,0.272727,Survival,none,0.0
1,0.023327,0.000242,0.392157,0.272727,Survival,none,0.0
2,0.025085,0.000242,0.039216,0.272727,Survival,none,0.0
3,0.023563,0.000242,0.411765,0.454545,Survival,none,0.0
4,0.024298,0.000242,0.137255,0.454545,Survival,none,0.0
...,...,...,...,...,...,...,...
1735835,0.025085,0.000504,0.019608,0.363636,Survival,none,0.0
1735836,0.023668,0.000324,0.352941,0.454545,Survival,none,0.0
1735837,0.024429,0.000319,0.352941,0.363636,Survival,none,0.0
1735838,0.045684,0.000300,0.098039,0.454545,Survival,none,0.0


In [5]:
X_train = df.drop(columns="type")
y_train = df["type"]

df = None # Release memory

# Split dataset into training and test data, stratify by the type of attack
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=0, shuffle=True, stratify=y_train)

# Use feature "Label" as classification label
d_temp: pd.DataFrame = pd.concat([X_train, y_train], axis="columns")
X_train, y_train = d_temp.drop(columns="Label"), d_temp["Label"]
d_temp: pd.DataFrame = pd.concat([X_test, y_test], axis="columns")
X_test, y_test = d_temp.drop(columns="Label"), d_temp["Label"]
d_temp = None # Release memory

In [6]:
rus = RandomUnderSampler(random_state=0)
X_train, y_train = rus.fit_resample(X_train, y_train)
bintr = np.bincount(y_train)
binte = np.bincount(y_test)
print(f"Labels\t\tTraining\tTesting\nNormal\t\t{bintr[0]}\t\t{binte[0]}\nAttack\t\t{bintr[1]}\t\t{binte[1]}")

Labels		Training	Testing
Normal		128320		465758
Attack		128320		54994


In [8]:
X_train.drop(columns=["type", "dataset"], inplace=True)
X_test.drop(columns=["type", "dataset"], inplace=True)

Deep Neural Network

In [10]:
import keras
from keras import layers
from compiledataset import tuner

# tuner.search_space_summary()
# tuner.search(X_train, y_train, epochs=2, validation_data=(X_test, y_test))
# tuner.results_summary()


dnn = keras.models.Sequential(
    [
        layers.Dense(10, activation='relu', input_dim=4),
        layers.Dense(10, activation='relu'),
        layers.Dense(10, activation='relu'),
        layers.Dense(10, activation='relu'),
        layers.Dense(10, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ]
)

dnn.compile(
    optimizer="adam",  # Optimizer rmsprop
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.Recall(), keras.metrics.FalseNegatives()],
)

X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

dnn.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=3,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_val, y_val),
)

results = dnn.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Epoch 1/3
7708/7708 [==============================] - 22s 3ms/step - loss: 0.1799 - binary_accuracy: 0.9211 - recall: 0.8975 - false_negatives: 12122.0000 - val_loss: 0.2051 - val_binary_accuracy: 0.9082 - val_recall: 0.9082 - val_false_negatives: 918.0000
Epoch 2/3
7708/7708 [==============================] - 20s 3ms/step - loss: 0.1610 - binary_accuracy: 0.9270 - recall: 0.9097 - false_negatives: 10687.0000 - val_loss: 0.2681 - val_binary_accuracy: 0.8585 - val_recall: 0.8585 - val_false_negatives: 1415.0000
Epoch 3/3
4069/4069 [==============================] - 8s 2ms/step - loss: 0.1380 - binary_accuracy: 0.9358 - recall: 0.9169 - false_negatives: 4569.0000
test loss, test acc: [0.138037770986557, 0.9358389377593994, 0.9169182181358337, 4569.0]
